In [1]:
import pandas as pd
import numpy as np

In [2]:
hcmst = pd.read_csv("hcmst2017.csv", encoding="mac_roman")

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (47,102) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cols = ["Q5", "Q21A_Year", "Q21A_Month", "Q21B_Year", "Q21B_Month", "Q21C_Year", "Q21C_Month", "Q21D_Year", "Q21D_Month"]
colnames = ["same_sex", "year_met", "month_met", "year_dating", "month_dating", "year_movein", "month_movein", "year_marry", "month_marry"]
relevant = hcmst[cols]
relevant.columns = colnames
relevant = relevant.loc[relevant["year_met"].notna()]
relevant.head()


,same_sex,year_met,month_met,year_dating,month_dating,year_movein,month_movein,year_marry,month_marry
1,NaN,1983,May,1995,August,1996,February,1996,February
2,NaN,2006,January,2006,June,2006,July,2008,May
4,NaN,1983,September,1983,October,1984,August,1984,August
5,"No, we are an opposite-sex couple",1981,December,1982,June,1982,June,1982,June
6,NaN,1966,October,1967,January,1969,August,1969,August


In [4]:
def month_year(row):
    year = row[0]
    if pd.isna(year) or year == "Refused":
        return np.nan
    month = "" if row[1] == "Refused" or pd.isna(row[1]) else row[1] + " "
    return month + year


In [5]:
clean = pd.DataFrame()
clean["same_sex"] = relevant["same_sex"].apply(
    lambda x: 0 if str(x).startswith("No") else -1 if pd.isna(x) else 0)
for stage in ["met", "dating", "movein", "marry"]:
    clean[stage] = relevant[["year_"+stage, "month_"+stage]].apply(month_year, axis=1)
clean.head()

,same_sex,met,dating,movein,marry
1,-1,May 1983,August 1995,February 1996,February 1996
2,-1,January 2006,June 2006,July 2006,May 2008
4,-1,September 1983,October 1983,August 1984,August 1984
5,0,December 1981,June 1982,June 1982,June 1982
6,-1,October 1966,January 1967,August 1969,August 1969


In [6]:
clean.to_csv("../data/time-data.csv", na_rep="null", index=False)